<a href="https://colab.research.google.com/github/rafi0486/Android-ui-templates/blob/master/Chatbot_2_workingwith%20less%20dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

def pprint(o):
    # small helper to make dict dumps a bit prettier
    print(json.dumps(o, indent=2))

####Install Rasa NLU and Rasa Core.

In [0]:
import sys
python = sys.executable

# In your environment run:
!{python} -m pip install -U rasa_core==0.9.6 rasa_nlu[spacy]===0.12.3;
!{python} -m pip install MySQL-python;
!{python} -m  pip install MySQL-python-connector;

# as well as install a language model:
!{python} -m spacy download en_core_web_md
!{python} -m spacy link en_core_web_md en --force;
!pip install mysql-connector-python




     |████████████████████████████████| 133kB 2.9MB/s 
     |████████████████████████████████| 133kB 39.5MB/s 
     |████████████████████████████████| 61kB 26.1MB/s 
     |████████████████████████████████| 71kB 22.6MB/s 
     |████████████████████████████████| 4.9MB 41.9MB/s 
     |████████████████████████████████| 51kB 21.5MB/s 
     |████████████████████████████████| 1.0MB 39.9MB/s 
     |████████████████████████████████| 307kB 35.7MB/s 
     |████████████████████████████████| 71kB 26.5MB/s 
     |████████████████████████████████| 665kB 29.2MB/s 
     |████████████████████████████████| 49.1MB 824kB/s 
     |████████████████████████████████| 81kB 24.9MB/s 
     |████████████████████████████████| 122kB 40.2MB/s 
     |████████████████████████████████| 81kB 27.1MB/s 
     |████████████████████████████████| 204kB 46.1MB/s 
     |████████████████████████████████| 3.1MB 29.5MB/s 
     |████████████████████████████████| 174kB 47.3MB/s 
     |████████████████████████████████| 3.1MB 28.7MB/s 

In [0]:
import rasa_nlu
import rasa_core
import spacy

print("rasa_nlu: {} rasa_core: {}".format(rasa_nlu.__version__, rasa_core.__version__))
print("Loading spaCy language model...")
print(spacy.load("en")("Hello world!"))

####Install some graph packages. You can jump this step if you have already installed it on your PC.

In [0]:
!apt-get -qq install -y graphviz libgraphviz-dev pkg-config;
!breq install graphviz
!{python} -m pip install pygraphviz;

## Part 1: Natural Language Understanding

Here some examples of the conversation with sentences from the user and exploited to build Rasa NLU model.  

### Creating the training data for language understanding model

In this step is originated a train set grouping user inputs into intents and highlighting key word by entities with its synonyms (entity values).  

In [0]:
nlu_md = """
## intent:greet
- hey
- hello there
- hi
- hello
- good morning
- good evening
- hey there
- let's go
- hey dude
- goodmorning
- goodevening
- good afternoon
- start
## intent:affirm
- great
- OK
- okay
- thank you
- thanks
- yes, thanks
- cheers
- thanks a lot
- thank you very much

## intent: health_about
- Please tell me about [health department](HealthInsurance),
- Please tell me the [health insurance](HealthInsurance) Plans for family
- What is [health insurance](HealthInsurance)
- For how many year's [health insurance](HealthInsurance) is covered
- Please tell me about [abc health insurance](HealthInsurance)
- Please tell me the [abc health insurance](HealthInsurance) Plans for family
- What is [abc health insurance](HealthInsurance)
- For how many year's [abc health insurance](HealthInsurance) is covered
- How to renew my [health insurance](HealthInsurance)
- What is [health insurance](HealthInsurance)
- I would like to know more about [health insurance](HealthInsurance)
- Please tell something about [health insurance](HealthInsurance)
- What is meant by [health insurance](HealthInsurance)
- Explain about [health insurance](HealthInsurance)
- Give me the details of [health insurance](HealthInsurance)
- Tell me about [health insurance](HealthInsurance)
- Please tell about [health insurance](HealthInsurance)
- How to renew my [abc health insurance](HealthInsurance)
- What is [abc health insurance](HealthInsurance)
- I would like to know more about [abc health insurance](HealthInsurance)
- Please tell something about [abc health insurance](HealthInsurance)
- What is meant by [abc health insurance](HealthInsurance)
- Explain about [abc health insurance](HealthInsurance)
- Give me the details of [abc health insurance](HealthInsurance)
- Tell me about [abc health insurance](HealthInsurance)
- Please tell about [abc health insurance](HealthInsurance)

## intent: vehicle_about
- How to renew my [abc vehicle insurance](VehicleInsurance)
- What is [abc vehicle insurance](VehicleInsurance)
- I would like to know more about [abc vehicle insurance](VehicleInsurance)
- Please tell something about [abc vehicle insurance](VehicleInsurance)
- What is meant by [abc vehicle insurance](VehicleInsurance)
- Explain about [abc vehicle insurance](VehicleInsurance)
- Give me the details of [abc vehicle insurance](VehicleInsurance)
- Tell me about [abc vehicle insurance](VehicleInsurance)
- Please tell about [abc vehicle insurance](VehicleInsurance)
- How to renew my [vehicle insurance](VehicleInsurance)
- What is [vehicle insurance](VehicleInsurance)
- I would like to know more about [vehicle insurance](VehicleInsurance)
- Please tell something about [vehicle insurance](VehicleInsurance)
- What is meant by [vehicle insurance](VehicleInsurance)
- Explain about [vehicle insurance](VehicleInsurance)
- Give me the details of [vehicle insurance](VehicleInsurance)
- Tell me about [vehicle insurance](VehicleInsurance)
- Please tell about [vehicle insurance](VehicleInsurance)


## intent: vehicle_benefits
- What are the [vehicle insurance](VehicleInsurance) [benefits](Benefits)?
- Are there any [vehicle insurance](VehicleInsurance) benefits?
- Tell me about the [benefits](Benefits) of  [vehicle insurance](VehicleInsurance)?
- What do you know about [vehicle insurance](VehicleInsurance) [benefits](Benefits)?
- What are the [benefits](Benefits) of taking ABC [vehicle insurance](VehicleInsurance)?
- What are the [benefits](Benefits) over other [vehicle insurance](VehicleInsurance) companies?
- What are the [abc vehicle insurance](VehicleInsurance) [benefits](Benefits)?
- Are there any [abc vehicle insurance](VehicleInsurance) [benefits](Benefits)?
- Tell me about the benifits of  [abc vehicle insurance](VehicleInsurance)?
- What do you know about [abc vehicle insurance](VehicleInsurance) [benefits](Benefits)?
- What are the [benefits](Benefits) of taking ABC [abc vehicle insurance](VehicleInsurance)?
- What are the [benefits](Benefits) over other [abc vehicle insurance](VehicleInsurance) companies?
- What are the [vehicle insurance](VehicleInsurance) [tax benefits](Benefits)?
- Are there any [vehicle insurance](VehicleInsurance) tax benefits?
- Tell me about the [tax benefits](Benefits) of  [vehicle insurance](VehicleInsurance)?
- What do you know about [vehicle insurance](VehicleInsurance) [tax benefits](Benefits)?
- What are the [tax benefits](Benefits) of taking ABC [vehicle insurance](VehicleInsurance)?
- What are the [tax benefits](Benefits) over other [vehicle insurance](VehicleInsurance) companies?
- What are the [abc vehicle insurance](VehicleInsurance) [tax benefits](Benefits)?
- Are there any [abc vehicle insurance](VehicleInsurance) [tax benefits](Benefits)?
- Tell me about the benifits of  [abc vehicle insurance](VehicleInsurance)?
- What do you know about [abc vehicle insurance](VehicleInsurance) [tax benefits](Benefits)?
- What are the [tax benefits](Benefits) of taking ABC [abc vehicle insurance](VehicleInsurance)?
- What are the [tax benefits](Benefits) over other [abc vehicle insurance](VehicleInsurance) companies?
- What are the [vehicle insurance](VehicleInsurance) [regular benefits benefits](Benefits)?
- Are there any [vehicle insurance](VehicleInsurance) regular benefits benefits?
- Tell me about the [regular benefits benefits](Benefits) of  [vehicle insurance](VehicleInsurance)?
- What do you know about [vehicle insurance](VehicleInsurance) [regular benefits benefits](Benefits)?
- What are the [regular benefits benefits](Benefits) of taking ABC [vehicle insurance](VehicleInsurance)?
- What are the [regular benefits benefits](Benefits) over other [vehicle insurance](VehicleInsurance) companies?
- What are the [abc vehicle insurance](VehicleInsurance) [regular benefits benefits](Benefits)?
- Are there any [abc vehicle insurance](VehicleInsurance) [regular benefits benefits](Benefits)?
- Tell me about the benifits of  [abc vehicle insurance](VehicleInsurance)?
- What do you know about [abc vehicle insurance](VehicleInsurance) [regular benefits benefits](Benefits)?
- What are the [regular benefits benefits](Benefits) of taking ABC [abc vehicle insurance](VehicleInsurance)?
- What are the [regular benefits benefits](Benefits) over other [abc vehicle insurance](VehicleInsurance) companies?



## intent: health_benefits
- What are the [health insurance](HealthInsurance) [benefits](Benefits)?
- Are there any [health insurance](HealthInsurance) [benefits](Benefits)?
- Tell me about the [benefits](Benefits) of  [health insurance](HealthInsurance)?
- What do you know about [health insurance](HealthInsurance) [benefits](Benefits)?
- What are the [benefits](Benefits) of taking ABC [health insurance](HealthInsurance)?
- What are the [benefits](Benefits) over other [health insurance](HealthInsurance) companies?
- What are the [abc health insurance](HealthInsurance) [benefits](Benefits)?
- Are there any [abc health insurance](HealthInsurance) [benefits](Benefits)?
- Tell me about the [benefits](Benefits) of  [abc health insurance](HealthInsurance)?
- What do you know about [abc health insurance](HealthInsurance) [benefits](Benefits)?
- What are the [benefits](Benefits) of taking ABC [abc health insurance](HealthInsurance)?
- What are the [benefits](Benefits) over other [abc health insurance](HealthInsurance) companies?
- What are the [health insurance](HealthInsurance) [regular benefits](Benefits)?
- Are there any [health insurance](HealthInsurance) [regular benefits](Benefits)?
- Tell me about the [regular benefits](Benefits) of  [health insurance](HealthInsurance)?
- What do you know about [health insurance](HealthInsurance) [regular benefits](Benefits)?
- What are the [regular benefits](Benefits) of taking ABC [health insurance](HealthInsurance)?
- What are the [regular benefits](Benefits) over other [health insurance](HealthInsurance) companies?
- What are the [abc health insurance](HealthInsurance) [regular benefits](Benefits)?
- Are there any [abc health insurance](HealthInsurance) [regular benefits](Benefits)?
- Tell me about the [regular benefits](Benefits) of  [abc health insurance](HealthInsurance)?
- What do you know about [abc health insurance](HealthInsurance) [regular benefits](Benefits)?
- What are the [regular benefits](Benefits) of taking ABC [abc health insurance](HealthInsurance)?
- What are the [regular benefits](Benefits) over other [abc health insurance](HealthInsurance) companies?
- What are the [health insurance](HealthInsurance) [complementary benefits](Benefits)?
- Are there any [health insurance](HealthInsurance) [complementary benefits](Benefits)?
- Tell me about the [complementary benefits](Benefits) of  [health insurance](HealthInsurance)?
- What do you know about [health insurance](HealthInsurance) [complementary benefits](Benefits)?
- What are the [complementary benefits](Benefits) of taking ABC [health insurance](HealthInsurance)?
- What are the [complementary benefits](Benefits) over other [health insurance](HealthInsurance) companies?
- What are the [abc health insurance](HealthInsurance) [complementary benefits](Benefits)?
- Are there any [abc health insurance](HealthInsurance) [complementary benefits](Benefits)?
- Tell me about the [complementary benefits](Benefits) of  [abc health insurance](HealthInsurance)?
- What do you know about [abc health insurance](HealthInsurance) [complementary benefits](Benefits)?
- What are the [complementary benefits](Benefits) of taking ABC [abc health insurance](HealthInsurance)?
- What are the [complementary benefits](Benefits) over other [abc health insurance](HealthInsurance) companies?


## intent: health_timeperiod
- What is the [validity](TimePeriod) of [abc health insurance](HealthInsurance)?
- What is the [duration](TimePeriod) of [abc health insurance](HealthInsurance)?
- How [long](TimePeriod)  is the [abc health insurance](HealthInsurance) valid?
- What is the [validity](TimePeriod) of [health insurance](HealthInsurance)?
- What is the [duration](TimePeriod) of [health insurance](HealthInsurance)?
- How [long](TimePeriod)  is the [health insurance](HealthInsurance) valid?
- What is the [validity](TimePeriod) of [health insurance](HealthInsurance)?
- What is the [duration](TimePeriod) of [health insurance](HealthInsurance)?
- How [long](TimePeriod)  is the [health insurance](HealthInsurance) valid?
- What is the [validity](TimePeriod) of [abc health insurance](HealthInsurance)?
- What is the [duration](TimePeriod) of [abc health insurance](HealthInsurance)?
- How [long](TimePeriod)  is the [abc health insurance](HealthInsurance) valid?


## intent: vehicle_timeperiod
- What is the [validity](TimePeriod) of [vehicle insurance](VehicleInsurance)?
- What is the [duration](TimePeriod) of [vehicle insurance](VehicleInsurance)?
- How [long](TimePeriod)  is the [vehicle insurance](VehicleInsurance) valid?
- What is the [validity](TimePeriod) of [abc vehicle insurance](VehicleInsurance)?
- What is the [duration](TimePeriod) of [abc vehicle insurance](VehicleInsurance)?
- How [long](TimePeriod)  is the [abc vehicle insurance](VehicleInsurance) valid?


## intent: vehicle_insuranceclaim
- What are the [documents](Documents) required for [vehicle insurance](VehicleInsurance)?
- What are the [documents](Documents) that i need to apply for [vehicle insurance](VehicleInsurance)?
- What all [documents](Documents) do i need for [vehicle insurance](VehicleInsurance)?
- WHAT [factors](Documents) DO I NEED TO CONSIDER BEFORE APPLYING FOR [vehicle insurance](VehicleInsurance)?
- What are the [documents](Documents) required for [abc vehicle insurance](VehicleInsurance)?
- What are the [documents](Documents) that i need to apply for [abc vehicle insurance](VehicleInsurance)?
- What all [documents](Documents) do i need for [abc vehicle insurance](VehicleInsurance)?
- WHAT [factors](Documents) DO I NEED TO CONSIDER BEFORE APPLYING FOR [abc vehicle insurance](VehicleInsurance)?

## intent: health_insuranceclaim
- What are the [documents](Documents) required for [health insurance](HealthInsurance)?
- What are the [documents](Documents) that i need to apply for [health insurance](HealthInsurance)?
- What all [documents](Documents) do i need for [health insurance](HealthInsurance)?
- WHAT [factors](Documents) DO I NEED TO CONSIDER BEFORE APPLYING FOR [health insurance](HealthInsurance)?
- What are the [documents](Documents) required for [abc health insurance](HealthInsurance)?
- What are the [documents](Documents) that i need to apply for [abc health insurance](HealthInsurance)?
- What all [documents](Documents) do i need for [abc health insurance](HealthInsurance)?
- WHAT [factors](Documents) DO I NEED TO CONSIDER BEFORE APPLYING FOR [abc health insurance](HealthInsurance)?


## intent: health_linking
- How to [link](Link) my aadhar?
- How to [link](Link) my pan card?
- How to [link](Link) my GST?
- Is it possible to [link](Link) my aadhar with my [health insurance](HealthInsurance)?
- Is it possible to [link](Link) my GST with my [health insurance](HealthInsurance)?
- Is it possible to [link](Link) my pan card with my [health insurance](HealthInsurance)?
- Is it necessasary to [link](Link) my aadhar with my [health insurance](HealthInsurance)?
- Is it necessasary to [link](Link) my GST with my [health insurance](HealthInsurance)?
- Is it necessasary to [link](Link) my pan card with my [health insurance](HealthInsurance)?
- How can I [link](Link) my aadhar with my [health insurance](HealthInsurance)?
- How can I [link](Link) my GST with my [health insurance](HealthInsurance)?
- How can I [link](Link) my pan card with my [health insurance](HealthInsurance)?
- How do I [link](Link) my aadhar?
- How do I [link](Link) my GST?
- How do I [link](Link) my pan card?


"""

%store nlu_md > nlu.md

### Defining the NLU model

With the training dataset is possible to build the NLU model by a pipeline which defines how structured data is extracted from unstructured user inputs. 

In [0]:
config = """
language: "en"
policies:
  - name: "KerasPolicy"
    featurizer:
    - name: MaxHistoryTrackerFeaturizer
      max_history: 5
      state_featurizer:
        - name: BinarySingleStateFeaturizer
  - name: "MemoizationPolicy"
    max_history: 5
pipeline:
- name: "nlp_spacy"                   # loads the spacy language model
- name: "tokenizer_spacy"             # splits the sentence into tokens
- name: "ner_crf"                     # uses the pretrained spacy NER model
- name: "intent_featurizer_spacy"     # transform the sentence into a vector representation
- name: "intent_classifier_sklearn"   # uses the vector representation to classify using SVM
- name: "ner_synonyms"                # trains the synonyms


""" 

%store config > config.yml

### Training the Rasa NLU Model

Let's start training the dataset with user inputs.

In [0]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# loading the nlu training samples
training_data = load_data("nlu.md")

# trainer to educate our pipeline
trainer = Trainer(config.load("config.yml"))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

### Using & evaluating the NLU model

Have a look on how the model is performing on a sentence:

In [0]:
pprint(interpreter.parse("What is coverage"))

Evaluation of a standalone model using all the dataset without a split train-test for semplicity. Upgraded version afford the split train-test option.

In [0]:
from rasa_nlu.evaluate import run_evaluation

run_evaluation("nlu.md", model_directory)


# Part 2: Handling the dialogue

In this part at the chatbot is taught how to make responses by training a dialogue management model with Rasa Core.

### Writing Stories

The dataset used for dialogue management models are represented by 'stories': actual conversations. User inputs expressed by intents, entities and value entities, start with '*'. Chatbot responses expressed as actions, start with '-'. 

In [0]:
stories_md = """

## path 0              
* greet
  - utter_greet             
* health_about{"HealthInsurance":"health insurance"}  
  - utter_action_health_about
* affirm  
  - utter_affirm
  
## path 1             
* greet
  - utter_greet             
* health_about{"HealthInsurance":"abc health insurance"}  
  - utter_action_health_about
* affirm  
  - utter_affirm  
  
  
## path 2           
* greet
  - utter_greet             
* vehicle_about{"VehicleInsurance":"vehicle insurance"}  
  - utter_action_vehicle_about
* affirm  
  - utter_affirm
  
## path 3           
* greet
  - utter_greet             
* vehicle_benefits{"VehicleInsurance":"vehicle insurance" and "Benefits":"benefits"}  
  - utter_action_vehicle_benefits
* affirm  
  - utter_affirm  
  
## path 4           
* greet
  - utter_greet             
* vehicle_benefits{"VehicleInsurance":"vehicle insurance" and "Benefits":"tax benefits"}  
  - utter_action_vehicle_benefits
* affirm  
  - utter_affirm    
  
  
## path 5           
* greet
  - utter_greet             
* vehicle_benefits{"VehicleInsurance":"vehicle insurance" and "Benefits":"regular benefits"}  
  - utter_action_vehicle_benefits
* affirm  
  - utter_affirm     
  
  
## path 6         
* greet
  - utter_greet             
* health_benefits{"HealthInsurance":"health insurance" and "Benefits":"benefits"}  
  - utter_action_health_benefits
* affirm  
  - utter_affirm  
    
## path 7        
* greet
  - utter_greet             
* health_benefits{"HealthInsurance":"health insurance" and "Benefits":"regular benefits"}  
  - utter_action_health_benefits
* affirm  
  - utter_affirm 
  
## path 8      
* greet
  - utter_greet             
* health_benefits{"HealthInsurance":"health insurance" and "Benefits":"complementary benefits"}  
  - utter_action_health_benefits
* affirm  
  - utter_affirm  
        
    
  

  
## path 1000  
* affirm
  - utter_affirm    
  
"""

%store stories_md > stories.md

### Defining a Domain

In the domain is specified the architecture of the dialogue with intents, entities, actions and also templates with answers used by the chatbot to respond to the user and slots which will help the chatbot to keep track of the context.

Here some links used as references for the answers:

https://en.wikipedia.org/wiki/List_of_emergency_telephone_numbers

https://exchange.aaa.com/travel-planning/international-travel/international-clubs/#.XM2ruXUzaV4


In [0]:
domain_yml = """
intents:
- greet
- health_about
- vehicle_about
- vehicle_timeperiod
- health_timeperiod
- vehicle_benefits
- health_benefits
- affirm


slots:
  HealthInsurance:
    type: categorical
    values:
        - health insurance
        - abc health insurance
  VehicleInsurance:
    type: categorical
    values:
        - vehicle insurance
        - abc vehicle insurance         
  Benefits:
    type: categorical
    values:
        - benefits
        - tax benefits
        - regular benefits
        - complementary benefits
  TimePeriod:
    type: categorical
    values:
        - timeperiod
        - duration
        - long
        - validity


  
entities:
- HealthInsurance
- VehicleInsurance
- Benefits
- TimePeriod


actions:
- utter_greet
- utter_affirm
- utter_action_health_about
- utter_action_vehicle_about
- utter_action_vehicle_benefits
- utter_action_health_benefits
- utter_action_vehicle_timeperiod
- utter_action_health_timeperiod
- action_default
- __main__.ApiAction

templates:
  utter_greet:
  - text: "Welcome"
  utter_action_health_about:
  - text: "It is health insurance, which can be covered for all medical day-care treatments such as skin treatment and Lasik surgery. The policy covered for 5 yrs. The Free look period is 15 days, the grace period and waiting period is 30 days."
  utter_action_vehicle_about:
  - text: "It is vehicle insurance, which can be covered for all own damage treatments such as fire, theft, accident. The policy covered for 5 yrs. The Free look period is 15 days, the grace period and waiting period is 30 days."
  utter_action_vehicle_benefits:
  - text: "a. Renewal Benefits :rn i. Cumulative Bonus (Additional Sum Insured) - An Additional Sum Insured of 10% of Annual Sum Insured provided on each renewal for every claim-free year up to a maximum of 50%.rnii. Complimentary vehicle service Coupons: One coupon per individual policy and two coupons per Floater policy will be offered.rn b. Tax Benefits: Income tax exemption for 1 lakh rupees"
  utter_action_health_benefits:
  - text: "a. Renewal Benefits :rni. a) Cumulative Bonus (Addits"
  utter_action_vehicle_timeperiod:
  - text: "The policy covered for 5 yrs"
  utter_action_health_timeperiod:
  - text: "The policy covered for 5 yrs"
  action_default:
  - text: "Default"
  utter_affirm:
  - text: "You're welcome"
  
"""

%store domain_yml > domain.yml

### Visualising the Training Data


In [0]:
from rasa_core.actions import Action
from rasa_core.events import SlotSet
from IPython.core.display import Image, display

import requests

class ApiAction(Action):
    def name(self):
        return "action_get_help"

    def run(self, dispatcher, tracker, domain):
        
        group = tracker.get_slot('Ambulance','Insurance','Fire','Police','Air_rescue','Road_help')
        
        r = requests.get('http://shibe.online/api/{}?count=1&urls=true&httpsUrls=true'.format(group))
        response = r.content.decode()
        response = response.replace('["',"")
        response = response.replace('"]',"")

#from IPython.display import Image
from rasa_core.agent import Agent


agent = Agent('domain.yml')
agent.visualize("stories.md", "story_graph.png", max_history=10)
Image(filename="story_graph.png")

[link text](https:// [link text](https://))### Training the Dialogue Model

This is a the training step of the dialogue management model with a neural network implemented in Keras splitting the dataset (90% train-10%test) and 50 epochs.

In [0]:
from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent


fallback = FallbackPolicy(fallback_action_name="action_default",
                          core_threshold=0.3,
                          nlu_threshold=0.3)
agent = Agent('domain.yml', policies=[MemoizationPolicy(max_history=5)])

# loading our neatly defined training dialogues
training_data = agent.load_data('stories.md',augmentation_factor=0)

agent.train(
    training_data,
    validation_split=0.1,
    epochs=50
)

agent.persist('models/dialogue')

In [0]:
from rasa_core.agent import Agent
agent = Agent.load('models/dialogue', interpreter=model_directory)

### Talking to the Bot 

In [0]:

# from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
# from rasa_core.agent import Agent


# fallback = FallbackPolicy(fallback_action_name="action_default",
#                           core_threshold=0.3,
#                           nlu_threshold=0.3)
# agent = Agent('domain.yml', policies=[MemoizationPolicy(max_history=5)])

# # loading our neatly defined training dialogues
# training_data = agent.load_data('stories.md',augmentation_factor=0)

# agent.train(
#     training_data,
#     validation_split=0.1,
#     epochs=50
# )
# agent.persist('models/dialogue')

In [0]:
from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent


agent = Agent('domain.yml', policies=[MemoizationPolicy(max_history=5)])

# loading our neatly defined training dialogues
training_data = agent.load_data('stories.md',augmentation_factor=0)

agent.train(
    training_data,
    validation_split=0.1,
    epochs=50
)
agent.persist('models/dialogue')

In [0]:
def getReply(msg_user):


  


  from rasa_core.agent import Agent
  agent1 = Agent.load('models/dialogue', interpreter=model_directory)
  print("Your bot is ready to talk! Start to say 'hello' or say 'stop' to finish or restart your dialogue")


  from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
  from rasa_core.agent import Agent

  i=0
  msgs=""
  invalidcntr=0
  while True:
      #msg_user=input()
      if(i==0):
        a="hello"
      else:
        a = msg_user
     # pprint(interpreter.parse(a))
      if(i>4):
        break;
 
     
      responses = agent1.handle_message(a)
          
      for response in responses:
            if(a=="start"):
              #msgs=response["text"]
              break;
            else:
              if(response["text"]!="Welcome"):
                msgs=msgs+str("<br/>")+response["text"]
      if a == 'stop':
          break         
              
     # if(invalidcntr==0 and i==0):
      #  msgs="Unable to understand the query.try something else."
        
      i+=1
  return msgs           
            

In [0]:
from IPython.display import clear_output

import mysql.connector
from mysql.connector.constants import ClientFlag

config = {
    'user': 'mrdevuj6_cmnuser',
    'password': '0486R@fika',
    'host': 'mrdev.co.in',
}
clear_output()
cnx = mysql.connector.connect(**config)
cur = cnx.cursor(buffered=True)
msg_txt=""
while True:
  cur.execute("select chat_id,lower(chat_msg) from mrdevuj6_devathon.chats where cstatus='P' limit 1;")
  rrow=cur.fetchone()
  if(rrow):
    msg_txt=rrow[1]
    print(msg_txt)
    rply=getReply(msg_txt)
    if(rply==""):
      rply="Cannot understand the query. Try something else."
    if(msg_txt=="start"):
      rply="Hello, welcome to ABC Insurance Virtual Assistant. How can I assist you today?";
    qry="update mrdevuj6_devathon.chats set chat_reply='"+str(rply)+"',cstatus='C' where  chat_id="+str(rrow[0])+";"
    #print(qry)
    cur.execute(qry)
    print(rply)
  if(msg_txt=="stop"):
    break;
      
   
#print(cur.fetchone())
cur.close()
cnx.close()

In [0]:
""
            
            
        